### Using MCP Tools in Llama Stack Client

In [ ]:
import os

from dotenv import load_dotenv
from llama_stack_client import LlamaStackClient, Agent, AgentEventLogger, RAGDocument
from termcolor import colored

In [ ]:
load_dotenv()

In [ ]:
base_url = f"http://{os.environ["LLAMA_STACK_SERVER_REMOTE_HOST"]}:{os.environ["LLAMA_STACK_SERVER_REMOTE_PORT"]}"

print(f"Connecting to {base_url}")

client = LlamaStackClient(
    base_url=base_url
)

In [ ]:
available_shields = [shield.identifier for shield in client.shields.list()]

if not available_shields:
    print(colored("No Shields are available", "yellow"))
else:
    print(f"Available Sheilds: {available_shields}")

In [ ]:
available_models = [model.identifier for model in client.models.list()]

print(f"Available Models: {available_models}")

In [ ]:
providers = client.providers.list()

for provider in providers:
    print(f"{provider.provider_id} (TYPE: {provider.provider_type})")

In [ ]:
tools = client.tools.list()

for tool in tools:
    print(f"{tool.identifier} - (Provider: {tool.provider_id})")

In [ ]:
from rich.pretty import pprint

pprint(client.tools.list(
    toolgroup_id = "mcp::ramkumar-weather"
))

In [ ]:
from rich.pretty import pprint

pprint(client.tools.list(
    toolgroup_id = "mcp::ramkumar-counter-r"
))

In [ ]:
from rich.pretty import pprint

pprint(client.tools.list(
    toolgroup_id = "mcp::ramkumar-file-system"
))

In [ ]:
from llama_stack_client import Agent, AgentEventLogger
from termcolor import cprint

In [ ]:
agent = Agent(
    client,
    model = available_models[0],
    instructions ="""You are a helpful assistant, who answers questions professionally.""",
    tools = [
        "mcp::ramkumar-counter-r",
        "mcp::ramkumar-weather",
        "mcp::ramkumar-file-system",
    ]
)

In [ ]:
user_prompts = [
    "Hello, How are you doing?",
    "what's written in the file /contents/foo?",
    "whats the current weather alerts in CA",
    "can you count how many r is found in RAMKUMAR?"
]

In [ ]:
session_id = agent.create_session("test-session")

print(f"Session {session_id} Created Successfully with the Agent {agent.agent_id}")

In [ ]:
for prompt in user_prompts:
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        session_id=session_id
    )

    for log in AgentEventLogger().log(response):
        log.print()